# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
#https://dato.com/download/
#!conda update pip
#!pip install --upgrade --no-cache-dir https://get.dato.com/GraphLab-Create/1.7.1/darrylbuswell@gmail.com/xxx/GraphLab-Create-License.tar.gz

import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('data/kc_house_data.gl/')

[INFO] 1449668138 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Anaconda3\envs\Python 2.7\lib\site-packages\certifi\cacert.pem
1449668138 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to darrylbuswell@gmail.com and will expire on December 04, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-24832 - Server binary: C:\Anaconda3\envs\Python 2.7\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\darry\AppData\Local\Temp\graphlab_server_1449668138.log.0
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt

sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before creating a new feature.
sales['floors'] = sales['floors'].astype(int) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 
                'bedrooms_square',
                'bathrooms',
                'sqft_living', 
                'sqft_living_sqrt',
                'sqft_lot', 
                'sqft_lot_sqrt',
                'floors', 
                'floors_square',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, 
                                              target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., 
                                              l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.177493     | 6981926.539042     | 427729.501760 |
PROGRESS: | 2         | 3        | 0.000002  | 1.193007     | 6861970.573528     | 393259.158552 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [6]:
model_all.get('coefficients').print_rows(num_rows=len(all_features) + 1)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274952.62044 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8483.95148798 |
|   sqft_living    |  None | 24.4238022551 |
| sqft_living_sqrt |  None | 351.097833343 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 850.427363977 |
|    sqft_above    |  None | 20.0777654516 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
import numpy as np # note this allows us to refer to numpy as np instead

In [9]:
l1_penalty_list = np.logspace(1, 7, num=13)
sort_table = []

for l1_penalty in l1_penalty_list:
    model = graphlab.linear_regression.create(training, 
                                              target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, 
                                              l1_penalty=l1_penalty, 
                                              verbose=False)
    predictions = model.predict(validation)
    RSS = sum([(predictions[i] - validation[i]['price']) ** 2 for i in range(len(predictions))])
    print('RSS:', RSS, 'penalty:', l1_penalty)
    sort_table.append((RSS, l1_penalty))
    
print(sorted(sort_table)[1])

('RSS:', 628412158085185.1, 'penalty:', 10.0)
('RSS:', 628412158282692.0, 'penalty:', 31.622776601683793)
('RSS:', 628412158907274.2, 'penalty:', 100.0)
('RSS:', 628412160882372.5, 'penalty:', 316.2277660168379)
('RSS:', 628412167128220.5, 'penalty:', 1000.0)
('RSS:', 628412186879698.1, 'penalty:', 3162.2776601683795)
('RSS:', 628412249343144.9, 'penalty:', 10000.0)
('RSS:', 628412446907721.2, 'penalty:', 31622.776601683792)
('RSS:', 628413072039906.6, 'penalty:', 100000.0)
('RSS:', 628415052663292.8, 'penalty:', 316227.76601683791)
('RSS:', 628421353760491.5, 'penalty:', 1000000.0)
('RSS:', 628441657747717.1, 'penalty:', 3162277.6601683791)
('RSS:', 628509646259482.2, 'penalty:', 10000000.0)
(628412158282692.0, 31.622776601683793)


In [10]:
best_model = graphlab.linear_regression.create(training, 
                                               target='price', 
                                               features=all_features,
                                               validation_set=None, 
                                               l2_penalty=0, 
                                               l1_penalty=10, 
                                               verbose=False)

best_model.get('coefficients').print_rows(num_rows=len(all_features) + 1)

+------------------+-------+------------------+
|       name       | index |      value       |
+------------------+-------+------------------+
|   (intercept)    |  None |  19772.5826717   |
|     bedrooms     |  None |  8163.51995199   |
| bedrooms_square  |  None |  963.286207169   |
|    bathrooms     |  None |  25578.4297787   |
|   sqft_living    |  None |  39.3451522891   |
| sqft_living_sqrt |  None |  1139.31384706   |
|     sqft_lot     |  None | 0.00596774968784 |
|  sqft_lot_sqrt   |  None |  151.205354426   |
|      floors      |  None |  20205.4712003   |
|  floors_square   |  None |  11652.1081691   |
|    waterfront    |  None |  602998.684348   |
|       view       |  None |  93445.2338191   |
|    condition     |  None |   6883.581066    |
|      grade       |  None |  6290.22186917   |
|    sqft_above    |  None |  43.6198492433   |
|  sqft_basement   |  None |  122.001332807   |
|     yr_built     |  None |  9.80274156192   |
|   yr_renovated   |  None |  57.3884930

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [11]:
predictions = best_model.predict(testing)
RSS = sum([(predictions[i] - testing[i]['price']) ** 2 for i in range(len(predictions))])
print(RSS)

1.57678098725e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [12]:
best_model['coefficients']['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [15]:
info = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, 
                                              target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, 
                                              l1_penalty=l1_penalty, 
                                              verbose=False)
    nnz = model['coefficients']['value'].nnz()
    info.append((l1_penalty, nnz))
    print('NZZ:', nnz, 'penalty:', l1_penalty)
    sort_table.append((nnz, l1_penalty))
    
print(sorted(sort_table)[1])

('NZZ:', 18, 'penalty:', 100000000.0)
('NZZ:', 18, 'penalty:', 127427498.57031322)
('NZZ:', 18, 'penalty:', 162377673.91887242)
('NZZ:', 18, 'penalty:', 206913808.11147901)
('NZZ:', 17, 'penalty:', 263665089.87303555)
('NZZ:', 17, 'penalty:', 335981828.62837881)
('NZZ:', 17, 'penalty:', 428133239.8719396)
('NZZ:', 17, 'penalty:', 545559478.11685145)
('NZZ:', 17, 'penalty:', 695192796.17755914)
('NZZ:', 16, 'penalty:', 885866790.41008317)
('NZZ:', 15, 'penalty:', 1128837891.6846883)
('NZZ:', 15, 'penalty:', 1438449888.2876658)
('NZZ:', 13, 'penalty:', 1832980710.8324375)
('NZZ:', 11, 'penalty:', 2335721469.0901213)
('NZZ:', 10, 'penalty:', 2976351441.6313128)
('NZZ:', 6, 'penalty:', 3792690190.7322536)
('NZZ:', 5, 'penalty:', 4832930238.5717525)
('NZZ:', 3, 'penalty:', 6158482110.6602545)
('NZZ:', 1, 'penalty:', 7847599703.5146227)
('NZZ:', 1, 'penalty:', 10000000000.0)
(1, 10000000000.0)


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [16]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[15]
print(l1_penalty_min, l1_penalty_max)

(2976351441.6313128, 3792690190.7322536)


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print(l1_penalty_values)

[  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [18]:
sort_table = []

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, 
                                              target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0, 
                                              l1_penalty=l1_penalty, 
                                              verbose=False)
    nnz = model['coefficients']['value'].nnz()
    if not nnz == max_nonzeros:
        continue
    predictions = model.predict(validation)
    RSS = sum([(predictions[i] - validation[i]['price']) ** 2 for i in range(len(predictions))])
    print('RSS:', RSS, 'penalty:', l1_penalty)
    sort_table.append((RSS, l1_penalty))
    
print(sorted(sort_table)[1])
minrss_penalty = sorted(sort_table)[1][1]

('RSS:', 1029979472296640.8, 'penalty:', 3320073020.20013)
('RSS:', 1032939019370513.6, 'penalty:', 3363038217.5212321)
('RSS:', 1036462853256876.0, 'penalty:', 3406003414.8423343)
('RSS:', 1040199358404093.5, 'penalty:', 3448968612.1634364)
('RSS:', 1045119929512428.0, 'penalty:', 3491933809.4845386)
('RSS:', 1050271584116935.1, 'penalty:', 3534899006.8056407)
('RSS:', 1055416895029827.8, 'penalty:', 3577864204.1267428)
('RSS:', 1060586344574354.9, 'penalty:', 3620829401.447845)
(1032939019370513.6, 3363038217.5212321)


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [19]:
best_model = graphlab.linear_regression.create(training, 
                                               target='price', 
                                               features=all_features,
                                               validation_set=None, 
                                               l2_penalty=0, 
                                               l1_penalty=minrss_penalty, 
                                               verbose=False)
best_model.get('coefficients').print_rows(num_rows=len(all_features) + 1)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 218017.932705 |
|     bedrooms     |  None | 1080.54982628 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None |  16464.698411 |
|   sqft_living    |  None | 32.9179739769 |
| sqft_living_sqrt |  None | 718.977806756 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 3089.53647721 |
|    sqft_above    |  None | 30.8142752399 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x